## 이미지 분류를 위한 버킷에 데이터 복사하고 레이블링하기

In [110]:
import os
%cd ~/
HOME=os.getcwd()

/home/jupyter


In [111]:
%cd dataset
!unzip dogs-vs-cats.zip

/home/jupyter/dataset
Archive:  dogs-vs-cats.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [112]:
!unzip train.zip

Archive:  train.zip
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [113]:
%cd train

/home/jupyter/dataset/train


In [114]:
BUCKET_NAME='kaggle-data-upload'
MOUNT_PATH ='/home/jupyter/bucket' 
!gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_PATH}

{"time":"27/11/2023 02:18:47.125064","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/jupyter/bucket\n"}
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithStorageHandle: Mount: mount: running /usr/bin/fusermount: exit status 1


In [115]:
train_size = 1000
valid_size = 500
test_size  = 500

train_index_end = train_size
valid_index_end = train_size + valid_size
test_index_end  = train_size + valid_size + test_size

## gcsfuse로 마운트하고 복사하기 : 4000장 이미지 복사하는데 5분 소요

## 그래서, vm에 small 데이터셋 디렉토리를 먼저 생성 후 디렉토리를 통채로 복사하는 것으로 실행

In [116]:
%%time
import os, shutil, pathlib
from tqdm.notebook import tqdm
from tensorflow.keras.utils import image_dataset_from_directory

IMG_SIZE = 224
BATCH_SIZE = 32

# 다운받은 데이터셋 디렉토리와 마운트된 버킷의 경로를 설정한다. 
original_dir = pathlib.Path(os.path.join(HOME,"dataset/train"))
# 로컬 디스크에 복사
new_base_dir = pathlib.Path(os.path.join(HOME, "dataset/cats_vs_dogs_small"))
#버킷에 직접 업로드
#new_base_dir = pathlib.Path(os.path.join(HOME, "bucket/cats_vs_dogs_small"))

def make_subset(subset_name, start_index, end_index):
    for category in tqdm(["cat", "dog"], desc='outer', position=0):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        
        for fname in tqdm(fnames, desc='inner', position=1):
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)
            
print("train_dataset")
make_subset("train", start_index=0, end_index=train_index_end)

print("valid_dataset")
make_subset("valid", start_index=train_index_end, end_index=valid_index_end)

print("valid_dataset")
make_subset("test", start_index=valid_index_end, end_index=test_index_end)


train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "valid",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

train_dataset


outer:   0%|          | 0/2 [00:00<?, ?it/s]

inner:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## gsutil로 복사하는데 48초 소요

In [100]:
GS_URI = 'gs://kaggle-data-upload'

In [102]:
%%time
!gsutil -m cp -r /home/jupyter/dataset/cats_vs_dogs_small/* gs://kaggle-data-upload

## 파일명으로 이미지 레이블링하고 데이터 프레임 만들기

In [104]:
import pandas as pd
from glob import glob
imgPath = '/home/jupyter/dataset/cats_vs_dogs_small/'
trainimages = glob(os.path.join(imgPath,"train/**/*.jpg"), recursive=True)
validimages = glob(os.path.join(imgPath,"valid/**/*.jpg"), recursive=True)
testimages = glob(os.path.join(imgPath,"test/**/*.jpg"), recursive=True)
allimages = trainimages + validimages + testimages
print(len(allimages))
bucketPath=[]
target=[]

for imageFile in allimages:
    class_name = os.path.basename(imageFile).split('.')[0]
    target.append(class_name)
    
    GS_URI = 'gs://kaggle-data-upload'
    bucketPath.append(GS_URI+ imageFile.split('small')[1]) 

len(target)
df = pd.DataFrame(data=bucketPath)
df['target'] = target
df.columns=['bucketPath','target']
df
#gs://kaggle-data-upload/train/dog/dog.0.jpg

4000


,bucketPath,target
0,gs://kaggle-data-upload/train/cat/cat.742.jpg,cat
1,gs://kaggle-data-upload/train/cat/cat.314.jpg,cat
2,gs://kaggle-data-upload/train/cat/cat.707.jpg,cat
3,gs://kaggle-data-upload/train/cat/cat.347.jpg,cat
4,gs://kaggle-data-upload/train/cat/cat.966.jpg,cat
...,...,...
3995,gs://kaggle-data-upload/test/dog/dog.1826.jpg,dog
3996,gs://kaggle-data-upload/test/dog/dog.1522.jpg,dog
3997,gs://kaggle-data-upload/test/dog/dog.1882.jpg,dog
3998,gs://kaggle-data-upload/test/dog/dog.1717.jpg,dog


In [108]:
df.to_csv('/home/jupyter/data.csv',index=False, header=False)

In [109]:
!cp /home/jupyter/data.csv bucket/data.csv

cp: cannot create regular file 'bucket/data.csv': No such file or directory
